In [18]:
import pandas as pd
import re


In [3]:
menu = pd.read_csv("../01_openrefine/Menu_clean.csv.gz", compression='gzip')
menu.fillna('', inplace=True)

In [4]:
menu.head()

,id,sponsor,event,venue,place,physical_description,occasion,notes,call_number,date,currency,currency_symbol,status,page_count,dish_count
0,12463,hotel eastman,breakfast,commercial,"hot springs, ar",CARD; 4.75X7.5;,easter,,1900-2822,15/04/1900,,,complete,2,67
1,12464,republican house,dinner,commercial,"milwaukee, wi",CARD; ILLUS; COL; 7.0X9.0;,easter,wedgewood blue card; white embossed greek key ...,1900-2825,15/04/1900,,,under review,2,34
2,12465,norddeutscher lloyd bremen,breakfast,commercial,dampfer kaiser wilhelm der grosse;,CARD; ILLU; COL; 5.5X8.0;,,"menu in german and english; illus, steamship a...",1900-2827,16/04/1900,,,complete,2,84
3,12466,norddeutscher lloyd bremen,lunch,commercial,dampfer kaiser wilhelm der grosse;,CARD; ILLU; COL; 5.5X8.0;,,"menu in german and english; illus, harbor scen...",1900-2828,16/04/1900,,,complete,2,63
4,12467,norddeutscher lloyd bremen,dinner,commercial,dampfer kaiser wilhelm der grosse;,FOLDER; ILLU; COL; 5.5X7.5;,,"menu in german and english; illus, harbor scen...",1900-2829,16/04/1900,,,complete,4,33


In [5]:
def freq(col):
    result = {}
    for row in col:
        if row in result: result[row] += 1
        else: result[row] = 1
    
    sort = []
    for (k, v) in result.items(): sort.append((k,v))
    sort = sorted(sort, key = lambda x: -x[1])
    for el in sort:
        print(str(el[0]) + "\t" + str(el[1]))
    return result

In [6]:
freq(menu['event'])

	9409
dinner	2218
breakfast	952
lunch	676
luncheon	547
daily menu	279
tiffin	174
supper	130
menu	118
annual dinner	106
banquet	75
dinner menu	56
daily	49
christmas dinner	49
annual banquet	46
bill of fare	45
fruhstuck	33
lunch or dinner	26
brunch	26
thanksgiving dinner	25
breakfast menu	23
carte du jour	21
daily menu, luncheon	21
breakfast or lunch	19
sunday dinner	18
dinner/diner	18
dinner, second cabin	17
luncheon/dejeuner fourchette	17
luncheon menu	15
thanksgiving day dinner	15
lunch & dinner	14
table d'hote dinner	14
wine list	14
daily bill of fare	14
second annual dinner	13
midday dinner	13
new year's day dinner	12
lunch a la carte	12
fifth annual dinner	11
first annual dinner	11
speisenfolge	11
third annual banquet	11
fourth annual dinner	10
daily menu, breakfast	10
lunch menu	10
fourth of july dinner	10
zweites fruhstuck	10
fruhstuck-luncheon	10
sixth annual banquet	10
ninth annual dinner	10
easter dinner	10
easter sunday dinner	10
supper menu	10
a la carte menu	9
evening meal	

{'breakfast': 952,
 'dinner': 2218,
 'lunch': 676,
 'supper': 130,
 'cafe luncheon': 2,
 'annual banquet': 46,
 'dinne': 1,
 '': 9409,
 'luncheon': 547,
 'carte du jour': 21,
 'annual meeting': 7,
 'banquet of the fifty ninth annual convention': 1,
 'fifth annual reunion': 2,
 'shakepeare commemoration': 1,
 '26th anniversary banquet': 1,
 'twenty-ninth annual dinner': 2,
 'sunday dinner': 18,
 'second annual dinner': 13,
 'seventh annual dinner': 4,
 'lunch & dinner': 14,
 'annual dinner': 106,
 'complimentary banquet given by the city government of boston to the boards of trade of the western cities': 1,
 'fest-bankett': 1,
 'daily menu': 279,
 'daily menu - 11;30 to 3;00': 1,
 'dinner to the board of officers of the catholic club of the city of new york': 1,
 'annual outing of edward thompson co.': 1,
 'banquet': 75,
 'menu': 118,
 'semi-annual dinner': 2,
 'souper': 3,
 '11th annual banquet': 2,
 'daily menu - 11;30 t0 3;00': 1,
 'a la carte menu': 9,
 'tiffin': 174,
 'annual outin

In [101]:
def mapEventToMeal(E):
    # Manual label overrides
    event = set(
        [
'paas festival',
'convention',
'wedding',
'shakepeare commemoration',
'personally conducted tour through mexico & california',
'avviversary of the boston massacre',
'special lenten dishes',
'41 jahres versammlung',
'54th annual regatta',
'examination paper',
'complimentary to mr. w.h.metson,prior to his departure for cape nome,alaska',
'new years eve',
'presidential tour to the pacific coast, april 29-june 15, 1901',
'pan-american restaurant co.',
'in honor of our veterans',
'fourth of july',
'fourth of july menu',
'quarterly membership meeting',
'en route to boston and return',
'presidential tour to the pacific coast',
'ninth annual lawn party',
'official visit of inspection of illustrious potentate charles a. benedict',
'inernational yacht races',
'see above',
'second meeting of above',
'9nth danzinger martini fest',
'specials of day',
'sunday tea',
'winkelried-auffuhrungen',
'15th annual conventin',
'erbsensuppe',
'menus for special train to episcopal convention in san francisco, oct. 2, 1901',
'goldenes jubilaeum (golden jubilee)',
'centennial celebration',
'lawn party & band concert',
'one hundred and fiftieth annivesary',
'pan american exposition',
'dibnner to edward m. grout',
'convention of superintendents',
'in honor of mr. hall caine',
'musicale & cotillion at cordova tonight',
'twenty-fifth annivesay o founding of club',
'second annual convention',
'vaudeville entertainment',
'30th annual meeting',
'eleventh annual convenion',
'20th annual meeting',
'in convivial session',
'ninth annual banquest and meeting',
'dedication',
'268 fall field day',
'inauguration day',
'thirteenth annual meeting',
'60 versammlung',
'liedertafel',
'in commemoration of the 73rd birthday of general grant',
'launching of the "saturday night"',
'saint nicholas day',
'to the captain & officers of h.n.m.ss van speijk',
"jefferson's birthday guest of honor william jennings bryan",
'thanksgiving day menu & program',
'easter sunday',
'tenth burns annivesary',
'dance',
'relig;',
'20th meeting',
'19th annual meeting',
'first meeting',
'general menu',
'peace jubilee',
'festessen zu ehren des 70 geburtstages von carl schurz',
'in honor of arthur henry meissiter,musical director',
'74th bithday of u.s. grant',
'excursion around the harbor of new york for the miller national association',
'cafe service',
'queen victoria''s birthday',
'banquest o the british fisheries commission',
'opening celebration for steamer sam sloan',
'st. patrick''s day',
'centennial celebration of the first settlement in ohio valley',
'trip to jamestown, va[ for hotel men''s mutual benefit association of the united states and canada',
'one hundred and third regular meeting',
'for the conseil federal suisse',
'127th annual meeting',
'new year buffet',
'17th annual festival',
'daily specials',
'probe-bankett',
'fauste nozze della caro figlia colomba rosa col sig. luigi profumo',
'directed by signor caioli luigi',
'extras for today menu;',
'annual festival',
'59th convention',
'opening of egyptian hall',
'national convention',
'testimonial to m.e. frederick e. barnes past grand high priest',
'old-fashioned rhode island clam bake',
'in commemoration of the birthday of abraham lincoln',
'bon voyage to mr. james pollock',
'twelfth annual meeting',
'opening of the hotel',
'fifty third annivrsary',
'to victor s. herbert & the pittsburgh orchestra',
'celebration of the evacuation of the city of new york',
'washingt0n''s birthday',
'picnic(?)',
'automobile night',
'53rd annual regatta',
'decimo aniversario de la revista de medicina y cerugia de la habana',
'94th anniv.',
'special menu',
'261st fall field day',
'no. iv of five subscription cotillions',
'presentation of sword to capt.a.c. braden',
'commemoration of the birthdaay of abraham lincoln',
'special dishes of to-day',
'arrival of the raleigh from manila',
'feast of saint john the evangelist',
'celebration of opening',
'in honour of national wholesale lumber dealers association',
'173rd regular meeting',
'fourth annual convention',
'mardi gras bal masque',
'banquest to ferdinand w. peck, commissioner of u.s.a. to paris exposition',
'stiftungsfest',
'florida golf association tournament',
'almuerzo',
'59th meeting',
'34th annual celebration',
'fourth annual meeting',
'20th annual social session, free to stockholders',
'clio menu',
'forefathers day',
'launching of ferryboat richmond',
'93rd regular meeting',
'126th annual meeting',
'raymond excursion party',
'launching of above',
'122nd annual meal',
'hunting menu to ministers of france',
'farewell to gen. horace porter, u.s. ambassador to the republic of france',
'atlas club minstrels',
'thanksgiving menu',
'96s decennial',
'wedding of wilhelmina zeisse and aloys schaufler',
'compl.',
'seventh annual festival',
'great surprise, the',
'box at ascot races',
'formal opening of the new house',
'meal for the members of terrace bowling club',
'daily menu, tea',
'collation to hotel men''s mutual benefit association',
'deed to h.m. howard',
'decennial celebration',
'second annual dance',
'opening collation',
'[general',
'lincheon',
'128th annual meeting',
'smoker',
'dining car service',
'annual outbreak',
'260th fall field day',
'dining room menu',
'governors'' day',
'3rd annual convention',
'102nd regular meeting',
'in honor of chief petty officers of h.m.s. drake',
'twenty- seventh annual meeting',
'dedication of mill of fall river iron works co. fall river, ma;',
'u.s.grant''s birthday',
'1899/11/09',
'natal day greeting',
'commemoration of the pilgrim''s statue unveiling',
'sechsten zusammenkunst',
'farewell to old delmonico''s',
'wedding day',
'hochzeitessen von herrn rechtsanwalt max guttmann und fraulein bertha lenpold',
'st. nicholas day, 1892',
'29th annual meeting',
'glefid',
'jubilee fete',
'complimentary to b.w. rowell, illustrious imperial recorder of imperial council of north america',
'semi-centennial of opening',
'symposium',
'lincoln night',
'visit of colonial prime ministers to the city of london',
'eleventh meeting',
'glorification oa william henry jackson',
'table d''hote de bon vivant',
'95th regular meeting',
'297th meeting',
'daily cafe menu',
'christmas dinnr',
'eighteenth annual convention',
'seventeenth annual social session',
'menu new year 1892',
'23rd annual meeting',
'collation on new year''s eve to his patrons',
'tariff',
'honoring james m. barrie and w. robertson nicoll',
'festessen zur feier des geburtstag sr.maj. kaiser wilhelm ii',
'thanksgiving day menu',
'1900s sexennial',
'opening day of new rooms',
'inauguration of mr. alderman alliston & lieut. col. clifford probyn, v.d.',
'meal for wedding celebration',
'258th fall field day',
'one hundredth aniversary of establishment of court',
'mayors'' day',
'second convention sales department',
'au revoir to mr. frank d. lalanne',
'reopening of club',
'ninth annual-in honor of crow charlton',
'souvenir menu',
'fifty-eighth meeting',
'commem. of evacuation of nyc by the british',
'third annual convention',
'schweinessen (pig eating?)',
'in commemoration of our late companion and commander general wallace',
'ramadhan snack menu',
'pre theatre',
'royal hotel white horse rangoon whiskey menu',
'inauguration of the oklahome chapter',
"queen victoria's birthday",
"[st. patrick's day?]",
"trip to jamestown, va[ for hotel men's mutual benefit association of the united states and canada]",
"daily menu, five o'clock tea",
"washingt0n's birthday",
"chef's tasting menu",
'"clio" menu'
"collation to hotel men's mutual benefit association",
'"deed to h.m. howard"',
'dinner', 'lunch', 'event',
"[general] grant's birthday",
'"annual outbreak"',
"governors' day",
"u.s.grant's birthday",
"commemoration of the pilgrim's statue unveiling",
"'farewell to old delmonico's",
'[birthday of princess thyra of denmark]',
"collation on new year's eve to his patrons",
"mayors' day",
"collation to hotel men's mutual benefit association",
"menu aboard ss barbarossa"])
    
    unk = "Unknown Meal for Event"
    
    # Convert to lower case and trim
    e = str(E).lower().strip()    
    result = []
    if re.search("(evening menu|dinner|diner|supper|dinne|fest-bankett|banquet|dnner|dinnwe|abendessen|dinhner|dinnner|evening meal|abend-essen|souper|hauptmahlzeit|banchetto)", e) != None: result.append("dinner")
    if re.search("(lunch|luncheon|dejeuner|mittagessen|mittagesses|midday meal|mittagesen|mitagessen|noonday meal|brunch|mittag-essen|daily menu - 11;30 to 3;00|daily menu - 11;30 t0 3;00|sandwich list|midday fare|a regular saturday afternoon deal)", e) != None: result.append("lunch")
    if re.search("(\btea\b|tiffin|tea menu|afternoon tea)", e) != None: result.append("tea")
    if re.search("(breakfast|brealfast|breakdast|breakast|fruhstuck|morning meal|fruhstuck|brunch|breakrfast)", e) != None: result.append("breakfast")
    if re.search("(dessert)", e) != None: result.append("dessert")
    if re.search("(wine list|beverage list|beverages|bar menu|cocktails|a selection of table wines|champagnes list|caviar and drink menu|drinks and snacks)", e) != None: result.append("wine list")        
    if re.search("(cafe \& restaurant menu|daily menu|\"clio\" menu|daily menus|daily menu and rates|^seafood$|^buffet$|day\'s menu|german\-african menu|complete menu|menu for all meals|spisenfolge|speisenfolge|regular menu|bill of fare|daily bill of fare|carte du jour|a la carte|speike-karte|speise-karte|speise karte|daily hotel menu|d\'hote|sunday menu|cafe service|dining car service meals|room service|third class menu|daily price list)", e) != None: result.append("menu")
    if re.search("(ball|reception|washington|anniversary|ladies night|reunion|outing|annual meeting)", e) != None: result.append("Unknown Meal for Event")
    if result == []:
        if e in event: result.append(unk)
        else: result.append(E)

    # Clean up where multiple patterns hit
    if "dinner" in result:
        if unk in result: result.remove(unk)
        if "menu" in result: result.remove("menu")
   
    if "lunch" in result:
        if unk in result: result.remove(unk)
        if "menu" in result: result.remove("menu")
    
    if "breakfast" in result:
        if unk in result: result.remove(unk)
        if "menu" in result: result.remove("menu")
            
    return str(result)

In [102]:
test = menu['event'].apply(mapEventToMeal)
freq(test)

['']	9409
['dinner']	4193
['lunch']	1501
['breakfast']	1093
['menu']	570
['Unknown Meal for Event']	381
['tea']	182
['lunch', 'breakfast']	57
['dinner', 'lunch']	51
['daily']	49
['wine list']	23
['dinner', 'breakfast']	16
['dinner', 'lunch', 'breakfast']	8
['dessert']	7
['lunch', 'tea']	2
['daily and wine']	2
['spiszeddel']	1


{"['breakfast']": 1093,
 "['dinner']": 4193,
 "['lunch']": 1501,
 "['']": 9409,
 "['menu']": 570,
 "['Unknown Meal for Event']": 381,
 "['dinner', 'lunch']": 51,
 "['tea']": 182,
 "['daily']": 49,
 "['dinner', 'breakfast']": 16,
 "['dinner', 'lunch', 'breakfast']": 8,
 "['wine list']": 23,
 "['lunch', 'breakfast']": 57,
 "['lunch', 'tea']": 2,
 "['spiszeddel']": 1,
 "['dessert']": 7,
 "['daily and wine']": 2}

In [88]:
freq(menu['venue'])

	9426
commercial	4705
soc	603
prof	437
restaurant	425
com	291
govt	169
railroad	155
educ	153
hotel,restaurant	124
hotel	122
nav	102
patr	102
mil	82
pol	80
foreign,restaurant	76
steamship	54
social	41
pat	35
other, private	29
gov	28
relig	26
?	21
airline	21
ss; for	21
greek letter (fraternity or sorority)	15
comm	13
rel;	10
polit;	10
foreign,hotel,restaurant	10
foreign	6
other (privately hosted dinner party)	6
other	6
other, personal	4
other private hosts	4
other (individual)	4
other, club	4
other,club	4
club	4
other (soc);	3
social club	3
other (group of friends);	3
naval	3
private	3
other (private party by single host)	2
private party	2
other (royal king);	2
patriotic	2
milit;	2
pol, soc	2
military	2
som;	2
educational	2
pro;	2
foreign,hotel	2
foreign,railroad	2
hotel; for	2
other (private party with individual host);	1
cam;	1
other: hospital (?);	1
other (sporting club);	1
other,(trade association)	1
other (special interest group);	1
other (merchant ship);	1
pricate	1
other (royal fi

{'commercial': 4705,
 '': 9426,
 'soc': 603,
 'educ': 153,
 'mil': 82,
 'govt': 169,
 'pol': 80,
 '?': 21,
 'restaurant': 425,
 'other, private': 29,
 'prof': 437,
 'relig': 26,
 'other (private party with individual host);': 1,
 'other (soc);': 3,
 'nav': 102,
 'com': 291,
 'other, personal': 4,
 'patr': 102,
 'cam;': 1,
 'other: hospital (?);': 1,
 'foreign': 6,
 'social': 41,
 'pat': 35,
 'railroad': 155,
 'other (sporting club);': 1,
 'other,(trade association)': 1,
 'rel;': 10,
 'other (private party by single host)': 2,
 'gov': 28,
 'other (special interest group);': 1,
 'other (merchant ship);': 1,
 'greek letter (fraternity or sorority)': 15,
 'polit;': 10,
 'pricate': 1,
 'hotel': 122,
 'social club': 3,
 'comm': 13,
 'private party': 2,
 'other (royal figure);': 1,
 'prog;': 1,
 'gk': 1,
 'other private hosts': 4,
 'other (individual)': 4,
 'other,(royal);': 1,
 'conn': 1,
 'other (privately hosted dinner party)': 6,
 'reporters of event': 1,
 'other (sport);': 1,
 'other (gr

In [135]:
def mapVenueToCleanVenue(x):
    other = set(["", "?", "null", "foreign", "other", "som;", "cam;", "conn", "dom;", "foreigneign", "unknown"])
    com = set(["com", "commercial", "comm", "com; pol", "com; soc", "commercoa;", "other: hospital (?);"])
    govt = set(["govt", "nav", "patr", "mil", "pol", "pat", "polit;", "naval", "milit;", "military",
               "other (royal king);", "other (british royal family);", "other (royal figure);",
               "other (royal host);", "other (royal monarch);", "soc, mil", "soc,pol", "soc;mil;",
               "political", "polit;", "naval", "patriotic", "milit;", "pol, soc", "military",
               'other (royal figure);', 'other (royal host);', 'other (head of royal family);',
               'other [?presumably british royal family];','soc,pol', 'soc, mil', 'other (royal monarch);',
               'other (royal king);', 'polit;', 'patr', 'pat', 'military', 'other (royal figure);',
               'other (british royal family);', 'soc;mil;', 'other (royal host);' ,'other (head of royal family);'])

    hotel = set(["hotel", "hotel,restaurant","hotel","foreign,hotel,rest","foreign,hotel","hotel; for", "foreign; hotel;",
                "hotel,other,ballroom", 'foreign,hotel,restaurant'])
    prof = set(["prof", "educ", "gov", "educational", "pro;", "educ; prof;", "edus;", "prof; com;",
                "prof;soc;", "prog;", "reporters of event"])
    rest = set(["restaurant", "resort", "restaurant,lunch room,coffee shop",
               "restaurant,other,urant,other,bar", "foreign,restaurant"])
    vessel = set(["railroad", "steamship", "airline", "ss; for", "foreign,railroad"])
    
    results = []
    if x in other: results.append("other")
    if x in com: results.append("com")
    if x in hotel: results.append("hotel")
    if x in prof: results.append("prof")
    if x in rest: results.append("restaurant")
    if x in vessel: results.append("vessel")
        
    if results == []: results.append("soc")
    return str(results)
    

In [138]:
test = menu['venue'].apply(mapVenueToCleanVenue)
freq(test)

['other']	9466
['com']	5011
['soc']	1421
['prof']	628
['restaurant']	504
['hotel']	262
['vessel']	253


{"['com']": 5011,
 "['other']": 9466,
 "['soc']": 1421,
 "['prof']": 628,
 "['restaurant']": 504,
 "['vessel']": 253,
 "['hotel']": 262}

In [89]:
freq(menu['place'])

	9422
new york	370
en route	296
delmonico's ny	117
en route aboard hong kong maru	101
ss "friedrich der grosse"	94
en route aboard ss. kasuga	89
?	85
ny	74
tampa, fl	64
schnelldampfer "auguste victoria"	55
ss city of para	50
st. augustine, fl	50
ss city of rio de janeiro	46
ss furst bismarck	42
ss sonoma	41
tampa,fla.	39
chicago	37
ss auguste victoria	34
an bord der "amerika"	34
schnelldampfer "hohenzollern"	34
ss westernland	33
en route aboard ss hong kong maru	32
ss barbarossa	29
miami biscayne bay, fl	28
dampfer "barbarossa"	27
ss doric	26
en route aboard sonoma	26
washington d.c.	25
saratoga springs, ny	25
en route aboard dampfer lahn	25
en route aboard express steamer furst bismarck	25
st.augustine,fla.	25
philadelphia	23
konigin luise	23
ss hohenzollern	23
en route aboard dampfer kronprinz wilhelm	22
at sea	22
tampa,fl	21
ss friesland	21
delmonico's new york	21
kaiser friedrich at sea	21
park lane	21
detroit, mi	20
narragansett pier r.i.	20
on board "new york"	20
en route friedri

{'hot springs, ar': 5,
 'milwaukee, wi': 8,
 'dampfer kaiser wilhelm der grosse;': 8,
 'r.m.s. "empress of china"': 9,
 'new york': 370,
 'schnelldampfer kaiser wilhelm der grosse;': 1,
 "delmonico's ny": 117,
 'ss city of para': 50,
 'ss doric': 26,
 '': 9422,
 "freemasons' tavern, [london, england]": 1,
 'brunswick hotel, boston, ma': 5,
 'en route aboard r.m.s. empress of china': 8,
 'en route aboard panama line steamship city of para': 9,
 'holland house, new york city': 1,
 'st. denis hotel (new york?)': 2,
 '(vienna, austria ?)': 1,
 "anderton's hotel, fleet street, e.c. (england?)": 1,
 'hoel flanders': 1,
 'german relief hall, mobile al': 1,
 'st. dennis hotel': 1,
 'en route aboard r.m.s. lucania': 8,
 '156 second avenue, ny': 2,
 "mouquin's": 1,
 'r.m.s. lucania': 14,
 "delmonico's": 17,
 'en route': 296,
 'ny': 74,
 '23rd st & bway ny': 1,
 '75 st. & columbus ave. ny': 1,
 "66 st. & b'way ny": 3,
 'hotel savoy,new york,ny': 2,
 'denver, colo.': 3,
 'boston, ma': 16,
 'der li

In [149]:
def cleanPlace(x, key):
    manualClean = {
    "new york":("New York City, New York, US","NA"),
"en route":("en route","Unknown"),
"delmonico's ny":("56 Beaver St, New York, NY 10004, US",""),
"en route aboard hong kong maru":("en route","Maru"),
"ss \"friedrich der grosse\"":("en route","SS friedrich der grosse"),
"en route aboard ss. kasuga":("en route","SS Kasuga"),
"?":("Unkown","NA"),
"ny":("New York City, New York, US","NA"),
"tampa, fl":("Tampa, Florida, US",""),
"schnelldampfer \"auguste victoria\"":("en route","SS Auguste Victoria"),
"ss city of para":("en route","SS City of Para"),
"st. augustine, fl":("St. Augustine, Florida, US","NA"),
"ss city of rio de janeiro":("en route","SS City of Rio de Janeiro"),
"ss furst bismarck":("en route","SS Furst Bismarck"),
"ss sonoma":("en route","SS Sonoma"),
"tampa,fla.":("Tampa, Florida, US","NA"),
"chicago":("Chicago, IL, US","NA"),
"ss auguste victoria":("en route","SS Auguste Victoria"),
"an bord der \"amerika\"":("en route","Amerika"),
"schnelldampfer \"hohenzollern\"":("en route","SS Hohenzollern"),
"ss westernland":("en route","Westernland"),
"en route aboard ss hong kong maru":("en route","Hong Kong Maru"),
"ss barbarossa":("en route","SS Barbarossa"),
"miami biscayne bay, fl":("Miami Biscayne Bay, Florida, US","NA"),
"dampfer \"barbarossa\"":("Miami Biscayne Bay, Florida, US","NA"),
"ss doric":("en route","SS Doric"),
"en route aboard sonoma":("en route","Sonoma"),
"washington d.c.":("Washington D.C., US","NA"),
"saratoga springs, ny":("Saratoga Springs, NY, US","NA"),
"en route aboard dampfer lahn":("en route","Dampfer Lahn"),
"en route aboard express steamer furst bismarck":("en route","Express Steamer Furst Bismarck"),
"st.augustine,fla.":("St. Augustine, Florida, US","NA"),
"philadelphia":("Philadelphia, PA, US","NA"),
"konigin luise":("en route","Konigin Luise"),
"ss hohenzollern":("en route","SS Hohenzollem"),
"en route aboard dampfer kronprinz wilhelm":("en route","Dampfer Kronprinz Wilhelm"),
"at sea":("en route","Unknown"),
"tampa,fl":("Tampa, Florida, US","NA"),
"ss friesland":("en route","SS Friesland"),
"delmonico's new york":("56 Beaver St, New York, NY 10004, US","NA"),
"kaiser friedrich at sea":("en route","Kaiser Friedrich"),
"park lane":("Unknown (Park Lane?)","NA"),
"detroit, mi":("Detroit, MI, US","NA"),
"narragansett pier r.i.":("Narragansett Pier, RI, US","NA"),
"on board \"new york\"":("en route","New York"),
"en route friedrich der grosse":("en route","Friedrich der Grosse"),
"rms campania":("en route","RMS Campania"),
"new york city":("New York City, New York, US","NA"),
"en route s.s. zeeland":("en route","SS Zeeland"),
"dining car":("en route","Unknown (Train)"),
"en route aboard ss. kamakura maru":("en route","SS Kamakura Maru"),
"southern pacific":("en route","Southern Pacific"),
"delmonico's":("56 Beaver St, New York, NY 10004, US","NA"),
"atlantic city, nj":("Atlantic City, NJ, US","NA"),
"ss coptic":("en route","SS Coptic"),
"london, england":("London, England","NA"),
"brooklyn, ny":("Brooklyn, NY, USA","NA"),
"aboard ss kamakura maru":("en route","SS Kamakura Maru"),
"en route aboard dampfer friedrich der grosse":("en route","Dampfer Friedrich der Grosse"),
"st.augustine,fl":("St. Augustine, Florida, US","NA"),
"boston, ma":("Boston, MA, US","NA"),
"ss gaelic":("en route","SS Gaelic"),
"cleveland, oh":("Cleveland, OH, USA","NA"),
"dampfer \"kronprinz wilhlem\"":("en route","Dampfer Kronprinz Wilhelm"),
"hotel manhattan, ny":("Hotel Manhattan, New York City, New York, US","NA"),
"rms oceanic":("en route","RMS Oceanic"),
"sherry's ny":("Sherry's (Unknown), New York City, NY","NA"),
"paris, france":("Paris, France","NA"),
"en route aboard ventura":("en route","Ventura"),
"the new willard, washington dc":("The New Willard, Washingon D.C., US","NA"),
"altoona, pa":("Altoona, PA, USA","NA"),
"en route aboard dampfer barbarossa":("en route","Dampfer Barbarossa"),
"r.m.s. lucania":("en route","RMS Lucania"),
"chicago, ill":("Chicago, IL, US","NA"),
"en route aboard ss kobe maru":("en route","SS Kobe Maru"),
"en route aboard ss.k0be maru":("en route","SS Kobe Maru"),
"en route s.s.zeeland":("en route","SS Zeeland"),
"schnelldampfer (express steamer) august victoria":("en route","August Victoria"),
"ss kronprinz wilhelm":("en route","SS Kronprinz Wilhelm"),
"steamship kaiser wilhelm der grosse":("en route","Kaiser Wilhelm der Grosse"),
"nassau,bahamas":("Nassau, Bahamas","NA"),
"gramercy park hotel":("Gramercy Park Hotel, Unknown","NA"),
"ss rio de janeiro":("en route","SS Rio de Janeiro"),
"u.s.m.s. new york":("en route","USMS New York"),
"st. louis, mo":("St. Louis, MO, US","NA"),
"jacksonville, fl":("Jacksonville, FL, US","NA"),
"quebec, canada":("Quebec, Canada","NA"),
"los angeles, ca":("Los Angelas, CA, US","NA"),
"waldorf-astoria, new york":("Waldorf-Astoria, New York City, New York, US","NA"),
"aboard ss kasuga;":("en route","SS Kasuga"),
"ss konigin luise":("en route","SS Konigin Luise"),
"unknown":("Unknown","Unknown"),
"coronado beach, ca":("Coronado Beach, CA, US","NA"),
"ellicott square, buffalo, ny":("Ellicott Square, Buffalo, NY, US","NA"),
"en route aboard s.s. westernland":("en route","SS Westernland"),
"old point comfort, va":("Old Point Comfort?, VA, US","NA"),
"tokyo, japan":("Tokyo, Japan","NA"),
"ss sierra":("en route","SS Sierra"),
"dining car service":("en route","Unknown (Train)"),
"ss kasuga":("en route","SS Kasuga"),
"san francisco":("San Francisco, CA, US","NA"),
"white mountains, nh":("White Mountains, NH, US","NA"),
"ss kaiser wilhelm der grosse":("en route","SS Kaiser Wilhel der Grosse"),
"auditorium hotel [chicago, il]":("Auditorium Hotel, Chicago, IL, USA","NA"),
"nassau, bahamas":("Nassau, Bahamas","NA"),
"manhattan hotel, new york, ny":("Manhattan Hotel, New York City, NY, USA","NA"),
"buffalo, ny":("Buffalo, NY","NA"),
"san francisco, ca":("San Francisco, CA, US","NA"),
"en route aboard ss friedrich der grosse":("en route","SS Friedrich der Grosse"),
"ss hong kong maru":("en route","Hong Kong Maru"),
"en route aboard furst bismarck":("en route","Furst Bismarck"),
"sunset ogden & shasta routes":("en route","Unknown (Sunset Ogden & Shasta Routes)"),
"palm beach, fl":("Palm Beach, FL, US","NA"),
"the waldorf-astoria":("The Waldorf-Astoria, New York City, NY, USA","NA"),
"providence":("Providence?","NA"),
"new orleans, la":("New Orleans, LA, US","NA"),
"sherry's, new york, ny":("Sherry's (Unknown), New York City, NY","NA"),
"waldorf-astoria":("The Waldorf-Astoria, New York City, NY, USA","NA"),
"en route - ss \"lahn\"":("en route","SS Lahn"),
"cincinnati, oh":("Cincinnati, OH, USA","NA"),
"clinton,ny":("Clinton, NY, USA","NA"),
"on board":("en route","Unknown (train)"),
"en route aboard dampfer h.h. meier":("en route","HH Meier"),
"ss deutschland":("en route","SS Deutschland"),
"schnelldampfer \"deutschland\" commodore albers":("en route","SS Deutschland"),
"en route aboard s.s. friesland":("en route","SS Friesland"),
"astor hotel, new york, ny":("Astor Hotel, New York City, NY, USA","NA"),
"lakewood, nj":("Lakewood, NJ, USA","NA"),
"park lane; 299 park avenue":("Park Lane, 299 Park Avenue, New York City, NY, USA","NA"),
"r.m.s. \"empress of china\"":("en route","RMS Empress of China"),
"en route aboard panama line steamship city of para":("en route","Panama Line City of Panama"),
"en route aboard konigin luise":("en route","Konigin Luise"),
"savoy hotel, new york, ny":("Savory Hotel, New York, NY, US","NA"),
"en route aboard s.s. zealandia":("en route","SS Zealandia"),
"montreal, canada":("Montreal, Canada","NA"),
"en route aboard kaiser wilhelm der grosse":("en route","Kaiser Wilhelm der Grosse"),
"en route aboard s.s. doric":("en route","SS Doric"),
"ritz carlton restaurant an bord der \"amerika\"":("en route","Amerika"),
"the auditorium [chicago, il]":("Auditorium Hotel, Chicago, IL, USA","NA"),
"en route aboard schnelldampfer auguste victoria":("en route","August Victoria"),
"concorde":("en route","Concorde"),
"san francisco overland limited":("en route","San Francisco Overland Limited"),
"dining car, southern pacific":("en route","Southern Pacific"),
"milwaukee, wi":("Milwaukee, WI","NA"),
"dampfer kaiser wilhelm der grosse;":("en route","Dampfer Kaiser Wilhelm der Grosse"),
"en route aboard r.m.s. empress of china":("en route","RMS Empress of China"),
"en route aboard r.m.s. lucania":("en route","RMS Lucania"),
"long beach, li [ny]":("Long Beach, New York City, New York","NA"),
"philadelphia, pa":("Philadelphia, PA, US","NA"),
"put-in-bay-island, lake erie, oh;":("Put-in-Bay Island, Lake Erie, OH, US","NA"),
"66th st and broadway,new york,ny":("66th st and Broadway, New York City, NY, US","NA"),
"ss st paul":("en route","SS St. Paul"),
"saratoga springs,ny":("Saratoga Springs, NY, US","NA"),
"sunset ogden & shasta route":("Unknown (Sunset Ogden & Shasta Route)","Unknown "),
"en route s.s.\"barbarossa\"":("en route","SS Barbarossa"),
"en route aboard dampfer friedrich der grosser":("en route","Dampfer Friedrich der Grosse"),
"en route aboard s.s.zealandia":("en route","SS Zealandia"),
"en route aboard s.s.zeeland":("en route","SS Zeeland"),
"[en route to canton?]":("en route","Unknown"),
"rome, italy":("Rome, Italy","NA"),
"aboard ss manitoba":("en route","SS Manitoba"),
"aboard kronprinzessin cecilie":("en route","Kronprinzessin Cecilie"),
"on board the ss vaderland":("en route","SS Vaderland"),
"portland, me":("Portland, ME, US","NA"),
"melbourne, australia":("Melbourne, Australia","NA"),
"nassau,n.p.bahamas":("Nassau, Bahamas","NA"),
"pullman dining car service en route":("en route","Unkown (Train)"),
"dampfer; h.h. meier":("en route","Dampfer H.H. Meier"),
"sonoma":("Sonoma??","NA"),
"nyc":("New York City, New York, US","NA"),
"en route s.s.vaderland":("en route","SS Vaderland"),
"clinton county, ny":("Clinton County, NY, USA","NA"),
"the pullman company - en route":("en route","Unknown (Train)"),
"aboard paquebot la lorraine":("en route","Paquebot la Lorraine"),
"en route aboard dampfer necker":("en route","Dampfer Necker"),
"ventura":("Ventura?","Unknown"),
"aboard paquebot la provence":("en route","Paquebot la Provence"),
"hotel astor":("Hoel Astor, New York, NY, US","NA"),
"on board s.s.kronprinzessin cecile":("en route","SS Kronprinzessin Cecile"),
"madison square, new york":("Madison Square, New York City, NY, US","NA"),
"[sr rr]enroute nashville to washington [dc]":("en route","Unknown"),
"nassau, n.p. bahamas":("Nassau, Bahamas","NA"),
"hotel brunswick ny":("Hotel Brunswick, New York City, NY","NA"),
"maison prunier":("Maison Prunier?","Unknown"),
"palace hotel":("Palace Hotel, San Francisco, CA","Unknown"),
"on board s.s. 'kronprinzessin cecilie'":("en route","SS Kronprinzessin Cecile"),
"r.m.s. etruria":("en route","RMS Etruria"),
"ss ventura;":("en route","SS Ventura"),
"queen city club [cincinnati, oh]":("Queen City Club, Cincinatti, OH, US","NA"),
"r.m.s. \"luncania\"":("en route","RMS Lucania"),
"hotel manhattan,ny":("Hotel Manhattan, New York City, New York, US","NA"),
"hotel savoy":("Hotel Savory","NA"),
"logan house, altoona, pa":("Logan House, Altoona, PA, US","NA"),
"en route aboard r.m.s. etruria":("en route","RMS Etruria"),
"on board ss north west":("en route","SS North West"),
"virginia beach, va":("Virgina Beach, VA, US","NA"),
"palace hotel, san francisco, ca":("Palace Hotel, San Francisco, CA","NA")}
    
    if x in manualClean: return manualClean[x][key]
    else: return "Unclean"

In [152]:
test1 = menu['place'].apply(lambda x: cleanLocation(x, 0))
test2 = menu['place'].apply(lambda x: cleanLocation(x, 1))
freq(test2)

Unclean	13550
NA	1659
Unknown	367
	181
SS Kasuga	114
Maru	101
SS friedrich der grosse	94
SS Auguste Victoria	89
SS City of Para	50
SS City of Rio de Janeiro	46
Hong Kong Maru	43
Amerika	43
SS Furst Bismarck	42
SS Sonoma	41
SS Zeeland	41
Dampfer Kronprinz Wilhelm	38
Unknown (Train)	37
SS Barbarossa	37
SS Doric	35
SS Kamakura Maru	35
SS Hohenzollern	34
Westernland	33
Konigin Luise	32
SS Friesland	31
RMS Lucania	28
SS Kobe Maru	28
Southern Pacific	27
Sonoma	26
Dampfer Lahn	25
Dampfer Friedrich der Grosse	25
Express Steamer Furst Bismarck	25
August Victoria	23
Kaiser Wilhelm der Grosse	23
SS Hohenzollem	23
Kaiser Friedrich	21
New York	20
Friedrich der Grosse	20
SS Deutschland	20
RMS Campania	19
RMS Empress of China	17
SS Coptic	17
SS Zealandia	17
SS Gaelic	16
RMS Oceanic	15
Ventura	15
SS Vaderland	15
Dampfer Barbarossa	15
SS Kronprinz Wilhelm	14
SS Kronprinzessin Cecile	14
SS Rio de Janeiro	13
USMS New York	13
SS Konigin Luise	13
RMS Etruria	12
SS Westernland	12
SS Sierra	12
SS Kaiser Wilh

{'Unclean': 13550,
 'NA': 1659,
 'Dampfer Kaiser Wilhelm der Grosse': 8,
 'RMS Empress of China': 17,
 '': 181,
 'SS City of Para': 50,
 'SS Doric': 35,
 'Panama Line City of Panama': 9,
 'RMS Lucania': 28,
 'Unknown': 367,
 'RMS Campania': 19,
 'Westernland': 33,
 'RMS Etruria': 12,
 'SS Ventura': 6,
 'Konigin Luise': 32,
 'SS Coptic': 17,
 'Maru': 101,
 'SS Rio de Janeiro': 13,
 'SS Westernland': 12,
 'USMS New York': 13,
 'SS City of Rio de Janeiro': 46,
 'Unknown (Train)': 37,
 'RMS Oceanic': 15,
 'SS Gaelic': 16,
 'SS North West': 6,
 'SS Sonoma': 41,
 'SS Zealandia': 17,
 'Unkown (Train)': 7,
 'SS Sierra': 12,
 'Dampfer H.H. Meier': 7,
 'SS St. Paul': 8,
 'SS Kasuga': 114,
 'SS Kobe Maru': 28,
 'Unknown ': 8,
 'Dampfer Lahn': 25,
 'SS Lahn': 10,
 'Ventura': 15,
 'SS Kamakura Maru': 35,
 'New York': 20,
 'SS Barbarossa': 37,
 'SS Zeeland': 41,
 'Dampfer Friedrich der Grosse': 25,
 'SS Friedrich der Grosse': 11,
 'SS Vaderland': 15,
 'SS friedrich der grosse': 94,
 'SS Furst Bismar

In [90]:
freq(menu['physical_description'])

	2782
BROADSHEET; ILLUS; 6.5 X 7.75;	127
21.5x13.5cm	118
33x18cm	116
28x18cm folded; 28x36cm open	109
21.5x14cm folded; 21.5x28cm open	87
33x21.5cm	79
28x18cm	76
28x21.5cm folded; 28x43cm open	70
CARD; ILLUS; COL; 5.25X8;	62
CARD; 5X8;	60
23x15cm folded; 23x30cm open	60
20.5x13cm	60
19x14cm folded; 19x28cm open	57
25.5x18cm folded; 25.5x36cm open	54
BROADSIDE; 5.5 X 7.75;	50
26.5x21cm folded; 26.5x42cm open	50
27x21cm folded; 27x42cm open	50
28x35.5cm	50
35.5x19.5cm folded; 35.5x39cm open	50
34.5x23cm	48
28x20.5cm folded; 28x41cm open	46
34x23cm	45
34x24cm folded; 34x48cm open	44
CARD; ILLUS; COL; 5X8;	43
24x16.5cm folded; 24x33cm open	41
CARD; COL; ILLUS; 5.5 X 8;	40
CARD; ILLUS; 5 X 8;	39
CARD;ILL;COL;5X8;	39
BROADSHEET; COL; ILLUS; 10.25 X 9.5;	39
19.5x14cm	39
CARD;ILL;COL;5.5X8.25;	38
33x23cm	38
24x15cm	38
24x16cm folded; 24x32cm open	37
28x17.5cm	37
23x15.5cm folded; 23x31cm open	34
30.5x23cm folded; 30.5x46cm open	33
28x21.5cm	33
29x21cm folded; 29x42cm open	32
22x14cm folded; 22

CARD; COL; 6 X 8;	1
BOOKLET; COL; ILLUS; 6.5 X 9.5;	1
FOLDER;ILL;COL;5X6.5;	1
BROADSIDE; 6X10;	1
BOOKLET;ILLUS;5.25X6.75;	1
BROADSHEET; 6.75 X 10;	1
BOOKLET; COL; ILLUS; 8.25 X 10.75;	1
BOOKLET; ILLUS; COL; 4.6X6.75;	1
CARD;4.5X6;	1
BOOKLET; COL; ILLUS; 9 X 4;	1
BOOKLET; ILLUS; 8X9.6;	1
BROADSHEET; 4.50 X 6.50;	1
CARD; ILLUS; 2.75 X 6;	1
BOOKLET;4 1/2X61/4	1
BROADSHEET; 3.5 x 5.5;	1
BOOKLET; ILLUS; 5.25X7.25;	1
BOOKLET; ILLUS; 5.5X7.5;	1
BOOKLET;4X7;	1
BOOKLET; ILLUS; COL; GOLD SEAL ON COVER;6.25 X 9.5;	1
FOLDER; ILLUS; 6.25 X 8.5;	1
Folder; 7.25 x 10 inches	1
Folder; 8.5 x 6 inches	1
Booklet; 9.75 x 12.5 inches	1
Booklet; 4.75 x 6.25 inches	1
Folded set of postcards; 7 x 4.5 in. folded; 7 x 30.5 in. extended	1
Tri-fold; 9.75 x 13.25 in. folded; 9.75 x 26 in. unfolded	1
Folder; 9.25 x 13 inches	1
Card; 4.5 x 7 inches	1
Folder; 9.5 x 12 inches	1
Folder; 10.75 x 14 inches	1
Card; 5.75 x 8.75 inches	1
Folder; 10 x 14.5 inches	1
Tri-fold; 5.75 x 11 inches folded	1
Folder; 8.75 x 14.5 inche

{'CARD; 4.75X7.5;': 4,
 'CARD; ILLUS; COL; 7.0X9.0;': 1,
 'CARD; ILLU; COL; 5.5X8.0;': 2,
 'FOLDER; ILLU; COL; 5.5X7.5;': 1,
 'CARD; ILLUS; COL; 4.75X7.25;': 3,
 'CARD; ILLUS; COL; 6.0X8.75;': 1,
 'BROADSIDE; ILLUS; COL; 5.5X8.50;': 2,
 'FOLDER; ILLUS; COL; 5.5X7.5;': 4,
 'CARD; ILLUS; COL; 4.25X5.5;': 1,
 'BOOKLET; ILLUS; COL; 5.5X7.0;': 1,
 'CARD; ILLUS; 6X9.5;': 2,
 'FOLDER; ILLUS; 6X9.25;': 1,
 'BROADSIDE; ILLUS; 5.5X8.5;': 19,
 '': 2782,
 'FOLDER; ILLUS; 5.5X8.5;': 8,
 'CARD; ILLUS; COL; 4.5X7;': 6,
 'CARD; 4.25X6.25;': 2,
 'FOLDER; ILLUS; COL; 4.5X6.5;': 5,
 'CARD; 4.5 X 7;': 24,
 'CARD; ILLUS; COL; 3.25 X 8.5;': 1,
 'FOLDER; 6 X 9;': 6,
 'BOOKLET; ILLUS; 5 X 7.25;': 1,
 'FOLDER; ILLUS; 4 X 5.5;': 1,
 'CARD; ILLUS; COL; 4.5 X 8;': 1,
 'CARD; ILLUS; 4.5 X 6.25;': 1,
 'BOOKLET, ILLUS; COL; 5 X 6.25;': 1,
 'BOOKLET; ILLUS; 4.5 X 6.5;': 3,
 'BOOKLET; ILLUS; 4 X 5.5;': 1,
 'CARD; 4.25 X 6.5;': 12,
 'CARD; COL; ILLUS; 5 X 7;': 3,
 'BROADSIDE; 6 X 9;': 6,
 'CARD;COL;4X6.5;': 1,
 'CARD;I

In [136]:
freq(menu['occasion'])

	13805
daily	1063
anniversary	609
complimentary/testimonial	569
daily menu	242
secular holiday	111
annual	97
religious holiday	87
patriotic holiday	75
other soc	64
other daily menu	59
rite	38
dinner	30
other anniversary	27
anniv	26
other daily	25
soc	24
annual dinner	23
compl	23
other	21
other anniv	19
pat	19
daily meal	18
other daily menu at sea	14
other reunion	13
other commemoration	11
other meeting	10
relig holiday	10
other social	9
easter	8
nat'l	8
eth	8
other daily hotel menu	7
rel	6
other special international event	6
ann	6
annual event	6
social	6
other annual event	6
daily dinner	6
other political	5
rite anniversary	5
aniv	5
luncheon	4
annual banquet	4
national event	4
breakfast	3
pol	3
tour to the pacific coast	3
reunion	3
other convention	3
anniversaryersary dinner	3
sunday	3
other annual meeting	3
banquet	3
sunday dinner	3
dinner en route	3
ethnic holiday	3
other,film premiere	3
complimentary dinner	2
ladies day	2
semi-annual	2
compl anniv	2
saturday dinner	2
complimentary	2

{'easter': 8,
 '': 13805,
 'anniversary': 609,
 'daily': 1063,
 'other complimentary': 1,
 'other daily': 25,
 'complimentary/testimonial': 569,
 'annual': 97,
 'other farewell': 1,
 'other annual dinner': 1,
 '78th birthday of u.s.grant': 1,
 'conducted tour': 1,
 'secular holiday': 111,
 'patriotic holiday': 75,
 'rite': 38,
 'evening dinner': 1,
 'other possibly a private dinner': 1,
 'breakfast': 3,
 'soc': 24,
 'daily menu': 242,
 'other meeting': 10,
 'religious holiday': 87,
 'pol': 3,
 'dinner': 30,
 'possibly a privately hosted party': 1,
 'festival': 1,
 'dailymenu': 1,
 'other political': 5,
 'tour to the pacific coast': 3,
 'complimentary dinner': 2,
 'buffet service': 1,
 'other,trolley party': 1,
 'other soc': 64,
 'compl. to staff': 1,
 'dining car serv': 1,
 'complimentary to commodore c. we. bliss': 1,
 'other reunion': 13,
 'reunion': 3,
 'other company dinner': 1,
 'ladies day': 2,
 'other see notes': 1,
 'avviv': 1,
 'relig & anniv': 1,
 '0ther commercial': 1,
 'oth

In [137]:
freq(menu['notes'])

	6933
dieter zander collection.	211
b&w photographs of various points of interest in america, hawaii, china;japan;	112
dinner	88
luncheon	88
english is secondary language	65
logo;	62
menu also contains english text	52
menu in french;	51
arthur h. rumpf menu collection	51
railroad dining car menu	45
priced menu;	44
english is secondary language.	44
flag;	42
menu handwritten;	40
hotel emblem;	38
hotel seal at top of menu;	37
handwritten; red border;	34
hotel seal at top;	33
french and english;	33
breakfast	33
wines listed;	29
handwritten; color illustraion of children carrying lanterns and large drum; map of steamship routes on back;	29
1 image	27
handwritten; green border;	25
lobby cafã©	25
menu in both german & english;	24
metropolitan life insurance company	24
menu also printed in french	24
english is the secondary language.	23
french line	23
3 images	22
the american committee	21
two copies;	20
includes wine list	20
decorative border;	19
hand written;	17
2 copies;	17
menu is ripped in

german; red and black print; club information handwritten;	1
separate english and german listing of menu;ship at sea with sailor hoisting flag in insert;musical program	1
red border; program included	1
wine list only;	1
[date info supplied by feb]; "30 cent table d'hote lunch & 40 cent table d'hote dinner - pint bottle of claret included with lunch & dinner;" a la carte menu in italian, remainder of menu in english; priced wine list on back of menu;	1
photograph of guest of honor on cover; drawings of greek mythical figures; silk ribbon attachments;	1
french; [sponsor & event info supplied by feb];	1
menu in german and english; picture ship passing rocky shore; concert program;	1
lists wines served with individual courses; illustration of female mariner on cover; [event & sponsor info supplied by fnb];	1
french; menu on front cover; toast listed innner back cover;	1
hand written; monogrammed; red seal;	1
titled reunion at norumbega	1
photo of cenonceaux and steamer; nautical flags;	1
w

{'': 6933,
 'wedgewood blue card; white embossed greek key border; "easter sunday" embossed in white; violet colored spray of flowers in upper left corner;': 1,
 'menu in german and english; illus, steamship and sailing vessel;': 1,
 'menu in german and english; illus, harbor scene with sailing vessel;': 1,
 'menu in german and english; illus, harbor scene with rocks and lighthouse; steamship and sailing vessels; concert program; dates: on german side of menu "montag, den 16 april 1900"; on english side of menu "monday, april 15th, 1900";': 1,
 'illus, red and white checkered flag;': 1,
 'hotel crest in blue; priced menu;': 1,
 'menu in german and english; illus, lighthouse; steamship; fishing dory;': 1,
 'menu in german and english; illus, sailing ship/sheets furled; steamship;': 1,
 'menu in german and english; illus, harbor; lighthouse; rocks; steamship; sailing ships; concert program;': 1,
 'hotel crest in blue;': 1,
 'vellum cover; crest of zeta psi; tied with blue satin ribbon; p

In [97]:
freq(menu['currency'])

	11089
Dollars	5549
Deutsche Marks	293
Francs	162
Canadian Dollars	98
Swiss Francs	47
Shillings	40
Swedish kronor (SEK/kr)	39
Italian Lire	28
Cents	24
UK Pounds	22
Belgian Francs	20
Mexican pesos	19
Dutch Guilders	17
Austrian Schillings	15
Danish kroner	14
Yen	9
Pesetas	7
Euros	7
Austro-Hungarian Kronen	5
Escudos	5
Hungarian forint	4
Drachmas	4
Israeli lirot (1948-1980)	3
Norwegian kroner	3
Icelandic KrÃ³nur	3
Quetzales	2
Argentine peso	2
Pence	1
MonÃ©gasque francs	1
Qatari riyal	1
Moroccan Dirham	1
Bermudian dollars	1
New Taiwan Dollar	1
Australian Dollars	1
Sol	1
Uruguayan pesos	1
Brazilian Cruzeiros	1
ZÅoty	1
Cuban pesos	1
Finnish markka	1
Lats	1
Straits dollar (1904-1939)	1


{'': 11089,
 'Dollars': 5549,
 'Cents': 24,
 'Francs': 162,
 'Belgian Francs': 20,
 'Shillings': 40,
 'Deutsche Marks': 293,
 'UK Pounds': 22,
 'Pence': 1,
 'Canadian Dollars': 98,
 'Austro-Hungarian Kronen': 5,
 'Swiss Francs': 47,
 'Pesetas': 7,
 'Danish kroner': 14,
 'Swedish kronor (SEK/kr)': 39,
 'Escudos': 5,
 'Yen': 9,
 'Italian Lire': 28,
 'Mexican pesos': 19,
 'Quetzales': 2,
 'Israeli lirot (1948-1980)': 3,
 'MonÃ©gasque francs': 1,
 'Qatari riyal': 1,
 'Dutch Guilders': 17,
 'Austrian Schillings': 15,
 'Euros': 7,
 'Norwegian kroner': 3,
 'Moroccan Dirham': 1,
 'Bermudian dollars': 1,
 'Hungarian forint': 4,
 'Drachmas': 4,
 'New Taiwan Dollar': 1,
 'Icelandic KrÃ³nur': 3,
 'Australian Dollars': 1,
 'Argentine peso': 2,
 'Sol': 1,
 'Uruguayan pesos': 1,
 'Brazilian Cruzeiros': 1,
 'ZÅ\x82oty': 1,
 'Cuban pesos': 1,
 'Finnish markka': 1,
 'Lats': 1,
 'Straits dollar (1904-1939)': 1}

In [128]:
freq(menu['currency_symbol'])

	11089
$	5579
DM	293
FF	162
C$	98
Fr	47
s	40
SEK	39
L	28
c	24
£	22
BEF	20
kr.	17
f	17
S	15
¥	9
Pt	7
€	7
K	5
 Ft	4
Drs.	4
I£	3
Kr.	3
Q	2
p	1
F	1
QR	1
Dh	1
NT$	1
S/.	1
$U	1
Cr.	1
zł.	1
mk	1
Ls	1


{'': 11089,
 '$': 5579,
 'c': 24,
 'FF': 162,
 'BEF': 20,
 's': 40,
 'DM': 293,
 '£': 22,
 'p': 1,
 'C$': 98,
 'K': 5,
 'Fr': 47,
 'Pt': 7,
 'kr.': 17,
 'SEK': 39,
 '¥': 9,
 'L': 28,
 'Q': 2,
 'I£': 3,
 'F': 1,
 'QR': 1,
 'f': 17,
 'S': 15,
 '€': 7,
 'Dh': 1,
 ' Ft': 4,
 'Drs.': 4,
 'NT$': 1,
 'Kr.': 3,
 'S/.': 1,
 '$U': 1,
 'Cr.': 1,
 'zł.': 1,
 'mk': 1,
 'Ls': 1}

In [98]:
freq(menu['status'])

complete	17371
under review	174


{'complete': 17371, 'under review': 174}

In [99]:
freq(menu['page_count'])

2	7472
4	3999
1	1748
3	1727
8	644
5	547
6	449
12	204
7	189
10	105
9	98
16	75
11	42
14	38
20	26
13	25
18	19
15	19
17	18
24	14
19	10
22	9
21	8
28	5
32	5
27	5
50	4
30	4
26	4
25	4
23	3
33	3
36	3
56	3
29	3
34	2
40	2
31	2
46	1
41	1
57	1
62	1
35	1
64	1
38	1
74	1


{2: 7472,
 4: 3999,
 12: 204,
 6: 449,
 10: 105,
 8: 644,
 3: 1727,
 7: 189,
 16: 75,
 34: 2,
 5: 547,
 24: 14,
 22: 9,
 14: 38,
 40: 2,
 18: 19,
 23: 3,
 20: 26,
 28: 5,
 1: 1748,
 32: 5,
 11: 42,
 50: 4,
 30: 4,
 33: 3,
 46: 1,
 36: 3,
 26: 4,
 15: 19,
 13: 25,
 56: 3,
 9: 98,
 19: 10,
 17: 18,
 41: 1,
 21: 8,
 57: 1,
 31: 2,
 25: 4,
 29: 3,
 27: 5,
 62: 1,
 35: 1,
 64: 1,
 38: 1,
 74: 1}

In [100]:
freq(menu['dish_count'])

18	511
21	464
20	460
16	454
24	422
17	421
22	412
19	400
15	362
14	356
13	352
23	349
26	339
25	322
28	313
12	296
11	273
30	269
27	238
29	234
10	222
32	208
31	187
34	182
36	163
38	147
9	141
35	139
33	135
40	127
49	127
39	124
8	124
37	123
44	120
45	120
46	113
48	113
41	111
43	110
50	107
47	104
51	97
54	97
42	95
72	87
52	86
7	83
62	81
53	80
55	79
60	79
69	79
56	75
61	71
58	69
64	69
57	69
67	68
68	68
66	67
70	66
63	63
6	61
59	60
71	57
80	56
65	55
82	53
81	52
78	52
74	50
100	49
99	48
84	46
97	46
98	46
87	45
73	45
76	45
83	45
101	45
90	43
88	43
85	43
94	41
93	40
102	40
104	40
77	40
86	39
75	39
144	38
5	38
105	38
89	37
133	37
79	37
96	36
95	36
174	36
138	35
91	35
92	34
137	34
125	34
143	34
157	34
188	34
103	33
106	33
114	33
120	33
0	32
112	32
142	32
110	31
130	31
187	31
139	31
177	30
141	30
140	30
123	29
122	29
172	29
156	29
154	29
173	29
108	28
136	28
161	28
131	27
113	26
164	26
124	26
107	26
134	26
115	25
166	25
111	25
163	25
119	25
175	25
129	24
145	24
121	24
109	24
171	24
126	24
116	24
176

{67: 68,
 34: 182,
 84: 46,
 63: 63,
 33: 135,
 37: 123,
 144: 38,
 80: 56,
 29: 234,
 19: 400,
 38: 147,
 176: 23,
 21: 464,
 24: 422,
 44: 120,
 42: 95,
 16: 454,
 129: 24,
 36: 163,
 15: 362,
 13: 352,
 14: 356,
 39: 124,
 22: 412,
 20: 460,
 17: 421,
 23: 349,
 97: 46,
 90: 43,
 30: 269,
 99: 48,
 203: 12,
 336: 5,
 392: 4,
 82: 53,
 56: 75,
 128: 23,
 35: 139,
 422: 1,
 475: 2,
 18: 511,
 12: 296,
 246: 14,
 28: 313,
 71: 57,
 93: 40,
 87: 45,
 55: 79,
 52: 86,
 53: 80,
 539: 2,
 113: 26,
 58: 69,
 152: 20,
 11: 273,
 110: 31,
 358: 5,
 72: 87,
 31: 187,
 0: 32,
 255: 8,
 60: 79,
 86: 39,
 66: 67,
 145: 24,
 177: 30,
 25: 322,
 45: 120,
 41: 111,
 121: 24,
 102: 40,
 103: 33,
 200: 15,
 342: 6,
 376: 4,
 81: 52,
 62: 81,
 123: 29,
 32: 208,
 403: 5,
 495: 4,
 254: 10,
 92: 34,
 96: 36,
 51: 97,
 50: 107,
 112: 32,
 40: 127,
 73: 45,
 250: 8,
 109: 24,
 46: 113,
 78: 52,
 389: 4,
 26: 339,
 242: 9,
 54: 97,
 70: 66,
 48: 113,
 191: 18,
 378: 8,
 27: 238,
 95: 36,
 43: 110,
 516: 2,

In [153]:
# Finally add the new columns to the data frame and write it back out as CSV
menu["placeClean"] = menu['place'].apply(lambda x: cleanLocation(x, 0))
menu["vesselName"] = menu['place'].apply(lambda x: cleanLocation(x, 1))
menu["meal"] = menu['event'].apply(mapEventToMeal)
menu["clanedVenue"] = menu['venue'].apply(mapVenueToCleanVenue)

In [157]:
menu.to_csv("p02_menu_cleaned.csv.gz", compression='gzip', index = False)